In [15]:
def regla_rectangulo(a,b,f):
    return (b-a)*f(a)

def regla_rectangulo2(x,y):

    if len(x) != 2:
        raise ValueError("R Rectangulo necesita dos puntos")
        
    n = len(x)
    base = x[n-1] - x[0]
    altura = y[0]
    return base*altura


def regla_trapecio(a,b,f):
    x1 = 0
    x2 = 0
    if a < b :
        x1 = a
        x2 = b
    else:
        x1 = b 
        x2 = a
    
    
    base_menor = f(x1)
    base_mayor = f(x2)
    altura = x2-x1

    return 0.5*(base_menor + base_mayor)*altura


def regla_trapecio2(x,y):

    if len(x) != 2 or len(y) != 2:
        raise ValueError("Trapecio simple necesita dos puntos")
        
    base_menor = y[0]
    base_mayor = y[1]
    altura = x[1] - x[0]

    return 0.5*(base_menor + base_mayor)*altura


def regla_simpson(a,b,f):
    c = (a + b) / 2
    k = (b-a)/6
    return (k*(f(a) + 4*f(c) + f(b)))

def regla_simpson2(x,y):
    
    if len(x) != 3 or len(y) != 3 :
        raise ValueError("Simpson simple necesita 3 punto")
        
    k = (x[2] - x[0]) / 6
    
    return k*(y[0] + 4*y[1] + y[2])


def regla_trapecio_compuesta(a,b,f,n):
    
    h = (b - a)/n
    suma = f(a) + f(b)
    for i in range (1,n-1):
        xi = a + i*h
        suma += 2*f(xi)
    
    return 0.5*h*suma

def regla_trapecio_compuesta2(x,y):
    n = len(x)
    h = (x[n-1] - x[0])/(n-1)

    suma = y[0] + y[n-1]

    for i in range(1,n-1):
        suma += 2*y[i]

    return 0.5*h*suma

def regla_simpson_compuesto(a,b,f,n):
    if n % 2 == 0:
        raise ValueError("SC necesita un numero de puntos impares")

    h = (b - a)/(n-1.0)
    suma = f(a) + f(b)
    
    for i in range(1,n-1):
        xi = a + i*h
        if i % 2 == 0:
            suma += 2*f(xi)
        else:
            suma += 4*f(xi)

    return (h/3)*suma

def regla_simpson_compuesto2(x,y):
    if len(x) % 2 == 0:
        raise ValueError("SC necesita un numero de puntos impares")

    n = len(x)
    h = (x[n-1] - x[0])/(n-1)
    suma = y[0] + y[n-1]
    
    for i in range (1,n-1):
        if i % 2 == 0:
            suma += 2*y[i]
        else:
            suma += 4*y[i]

    return ((h/3.0)*suma)

In [17]:
import numpy as np

x = np.array([0,100,200,300,400,500,600,700,800,900,1000])
y = np.array([125,125,120,112,90,90,95,88,75,35,0])

i_sc = regla_simpson_compuesto2(x,y)
i_tc = regla_trapecio_compuesta2(x,y)
print(f"i_sc = {i_sc}")
print(f"i_tc = {i_tc}")

i_sc = 89500.0
i_tc = 89250.0


In [16]:
import sympy as sp
import numpy as np
from scipy.interpolate import CubicSpline

def diferencias_divididas(x,y):
    d = y.copy()
    n = len(x)
    for i in range(1,n):
        for j in range(n-1,i-1,-1):
            d[j] = (d[j] - d[j-1])/(x[j] - x[j-i])

    return d

def metodo_newton(x,y):
    n = len(x)
    d = diferencias_divididas(x,y)
    suma = d[0]
    prod = 1
    X = sp.Symbol('x')
    
    for i in range(1,n):
        prod *= (X - x[i - 1])
        suma += d[i]*prod

    return sp.lambdify(X,suma)


def metodo_richardson(x,y):
    a = x[0]
    b = x[-1]
    n = len(x)
    f = CubicSpline(x,y)
    I = np.zeros(2)
    
    for i in range(2):
        m = (n-1)*(2**i)
        h = (b - a)/m
        suma = f(a) + f(b)
        for j in range(1,m):
            xj = a + j*h
            suma += 2*f(xj)
            
        I[i] = (h/2.0)*suma
        
        
    return ((4/3.0)*I[1]) - ((1/3.0)*I[0])

x = np.array([0,100,200,300,400,500,600,700,800,900,1000])
y = np.array([125,125,120,112,90,90,95,88,75,35,0])

print(metodo_richardson(x,y))

    

89434.27835051547


In [61]:
#I00
#I10 I11
#I20 I21 I22
#I30 I31 I32 I33
import numpy as np

def metodo_romberg(a, b, f, n=10, eps = 10**(-3)):
    R = np.zeros((n,n))
    h = b-a
    R[0,0] = 0.5*h*(f(a) + f(b))

    for j in range(1,n):
        suma = 0
        m = 2**(j-1) + 1
        for i in range(1, m):
            xi = a + (i - 0.5)*h
            suma += f(xi)

        R[j,0] = 0.5*(R[j-1,0] + suma*h)

        for k in range(1, j+1):
            factor = 4**k
            R[j,k] = ((factor*R[j,k-1]) - R[j-1,k-1])/(factor - 1)

        if abs(R[j,j] - R[j-1,j-1]) < eps : 
            print(f"Convergencia alcanzada en {j} iteraciones")
            return R[j,j], R[:j+1,:j+1]

        h = h/2

    print("Numero maximo de iteraciones alcanzadas")
    return R[n-1,n-1], R[:n, :n]

        
a = 0
b = 0.8
def f(x):
    return 0.2 + 25*x - 200*x**2 + 675*x**3 - 900*x**4 + 400*x**5

n = 10
eps = 10**(-3)
I, R =  metodo_romberg(a, b, f,n, eps)

print(I)
print()
print(R)


















Convergencia alcanzada en 3 iteraciones
1.6405333333333363

[[0.1728     0.         0.         0.        ]
 [1.0688     1.36746667 0.         0.        ]
 [1.4848     1.62346667 1.64053333 0.        ]
 [1.6008     1.63946667 1.64053333 1.64053333]]


In [33]:
#- + +
import numpy as np

def metodo_gl(a,b,f,n):
    nodos, pesos = np.polynomial.legendre.leggauss(n)
    t = 0.5*(b-a)*nodos + 0.5*(b+a)
    w = 0.5*(b-a)*pesos
    I = np.sum(w*f(t))
    return I
    

In [67]:
#I00
#I10 I11
#I20 I21 I22
#I30 I31 I32 I33

import numpy as np
def romberg_while(a,b,f,n = 10,eps = 10**(-3)):
    I = np.zeros((n,n))
    h = b-a
    I[0,0] = 0.5*h*(f(a) + f(b))

    j = 1
    error = 1
    while j < n and error > eps :
        suma = 0
        m = 2**(j-1) + 1
        for i in range(1,m):
            xi = a + (i-0.5)*h
            suma += f(xi)

        I[j,0] = 0.5*(I[j-1,0] + h*suma)

        for k in range(1,j+1):
            factor = 4**k
            I[j,k] = ((factor*I[j,k-1]) - I[j-1,k-1])/(factor - 1)
            
        error = abs(I[j,j] - I[j-1,j-1])
        h /= 2
        j +=1
        

    if(j >= n):
        print("Numero de interacion maximo alcanzado")
        return I[j-1,j-1], I[:n-1,:n-1]


    print(f"Convergencia alcanzada en la iteracion {j}")
    return I[j-1,j-1], I[:j,:j]

a = 0
b = 0.8
def f(x):
    return 0.2 + 25*x - 200*x**2 + 675*x**3 - 900*x**4 + 400*x**5

n = 5
eps = 10**(-3)
I, R =  romberg_while(a, b, f,n, eps)

print(I)
print()
print(R)
        
        

Convergencia alcanzada en la iteracion 4
1.6405333333333363

[[0.1728     0.         0.         0.        ]
 [1.0688     1.36746667 0.         0.        ]
 [1.4848     1.62346667 1.64053333 0.        ]
 [1.6008     1.63946667 1.64053333 1.64053333]]
